# Check for CW Task Compliance.ipynb

Check for an exact match to the prompted sentences.

Search all responses - odd conversation indices.

First sentence plus "\n" at the end.

Second sentence plus end of response/end of trimmed string.

In [1]:
import pickle
from Grading_Functions import *
from Evaluation_Functions import *
import pandas as pd
from unidecode import unidecode

# Load conversations
with open('Conversation_Lists.pkl', 'rb') as file:
    loaded_data = pickle.load(file)


In [2]:
mtm_list = [mtm for mtm in loaded_data.keys()]
conversations_list = [conversation for conversation in loaded_data.values()]

print(len(mtm_list))
print(len(conversations_list))


32
32


In [3]:
# Iterate over elements of dictionary loaded_data
# Load the list at that element
# Each element in that list is a list (conversation)
# Save fill lists to store model_task_method and the conversation number
model_task_methods = []
conversation_numbers = []
flat_conversations = []
for model_task_method, conversations in zip(mtm_list, conversations_list):
    for i, conversation in enumerate(conversations):
        model_task_methods.append(model_task_method)
        conversation_numbers.append(i+1)
        # Convert all conversation elements to strings
        all_string_conversation = [str(element) for element in conversation]
        flat_conversations.append(all_string_conversation)


In [4]:
print(conversations_list[0][0])


["Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nA:", "She makes $6 every day at the farmers' market."]


In [5]:
# Load in text file data_100_random_text.txt
with open('Creative Writing/data_100_random_text.txt', 'r') as file:
    data = file.read()

# Get lines
lines = data.split('\n')

# Split each line on periods
sentences = [line.split('.') for line in lines]

# Keep first two sentences
first_two = [sentence[0:2] for sentence in sentences]

# Strip leading and trailing blanks and add periods back
first_two_clean = [[sentence.strip() + '.' for sentence in sentence_pair] for sentence_pair in first_two]

# Use unidecode to convert to ascii
first_two_clean = [[unidecode(sentence) for sentence in sentence_pair] for sentence_pair in first_two_clean]

# Append 1., 2. to the beginning of each sentence
# numbered_sentences = []
# for pair in first_two_clean:
#     numbered_sentences_pair = []
#     for index in range(len(pair)):
#         numbered_sentences_pair.append(str(index + 1) + '. ' + pair[index])
#     numbered_sentences.append(numbered_sentences_pair)
    
# print(numbered_sentences)


In [6]:
# Create cw_conversations_list
cw_model_task_methods = []
cw_flat_conversations_no_metadata = []
cw_conversation_numbers = []
cw_sentences = []
for index in range(len(flat_conversations)):
    if 'cw' in model_task_methods[index]:
        if 'gpt4' in model_task_methods[index]:
            cw_model_task_methods.append(model_task_methods[index])
            cw_flat_conversations_no_metadata.append(gpt4_cw_cleanup(flat_conversations[index]))
            cw_conversation_numbers.append(conversation_numbers[index])
            cw_sentences.append(first_two_clean[conversation_numbers[index]-1])
        else:
            cw_model_task_methods.append(model_task_methods[index])
            cw_flat_conversations_no_metadata.append(td3_cw_cleanup(flat_conversations[index]))
            cw_conversation_numbers.append(conversation_numbers[index])
            cw_sentences.append(first_two_clean[conversation_numbers[index]-1])


In [7]:
# Use the compliance checker
compliance_list = []
#for index in range(1000, 1001):
for index in range(len(cw_flat_conversations_no_metadata)):
    # Feed numbered sentences
    # If self_refine or tree_of_thought is in the model_task_method, feed the third to last message
    # Otherwise feed the last message
    # print(cw_sentences[index][0])
    # print(cw_sentences[index][1])
    # print(cw_model_task_methods[index])
    # print(cw_flat_conversations_no_metadata[index][-1])
    # print(bool(re.search(cw_sentences[index][0], cw_flat_conversations_no_metadata[index][-1])))
    # print(bool(re.search('\\n', cw_flat_conversations_no_metadata[index][-1])))
    # print(cw_flat_conversations_no_metadata[index][-1].find(cw_sentences[index][0] + '\\n') > 0)
    # print(bool(re.search(cw_sentences[index][0] + '\\n', cw_flat_conversations_no_metadata[index][-1])))
    # print(bool(re.search(cw_sentences[index][1] + '$', cw_flat_conversations_no_metadata[index][-1])))
    if 'gpt4' in cw_model_task_methods[index]:
        if 'self_refine' in cw_model_task_methods[index] or 'tree_of_thought' in cw_model_task_methods[index]:
            if (bool(cw_flat_conversations_no_metadata[index][-3].find(cw_sentences[index][0] + '\\n') > 0) or bool(cw_flat_conversations_no_metadata[index][-3].find(cw_sentences[index][0] + ' \\n') > 0)) and (bool(re.search(cw_sentences[index][1] + '$', cw_flat_conversations_no_metadata[index][-3])) or bool(re.search(cw_sentences[index][1] + '\n$', cw_flat_conversations_no_metadata[index][-3]))):
                compliance_list.append(1)
            else:
                compliance_list.append(0)
            #compliance_list.append(grade_creative_writing_task_adherence(cw_sentences[index][0], cw_sentences[index][1], cw_flat_conversations[index][-3]))
        else:
            #compliance_list.append(grade_creative_writing_task_adherence(cw_sentences[index][0], cw_sentences[index][1], cw_flat_conversations[index][-1]))
            if (bool(cw_flat_conversations_no_metadata[index][-1].find(cw_sentences[index][0] + '\\n') > 0) or bool(cw_flat_conversations_no_metadata[index][-1].find(cw_sentences[index][0] + ' \\n') > 0)) and (bool(re.search(cw_sentences[index][1] + '$', cw_flat_conversations_no_metadata[index][-1])) or bool(re.search(cw_sentences[index][1] + '\n$', cw_flat_conversations_no_metadata[index][-1]))):
                compliance_list.append(1)
            else:
                compliance_list.append(0)
    else:
        if 'self_refine' in cw_model_task_methods[index] or 'tree_of_thought' in cw_model_task_methods[index]:
            if (bool(re.search(cw_sentences[index][0] + '\n', cw_flat_conversations_no_metadata[index][-3])) or bool(re.search(cw_sentences[index][0] + ' \n', cw_flat_conversations_no_metadata[index][-3]))) and bool(re.search(cw_sentences[index][1] + '$', cw_flat_conversations_no_metadata[index][-3])):
                compliance_list.append(1)
            else:
                compliance_list.append(0)
            #compliance_list.append(grade_creative_writing_task_adherence(cw_sentences[index][0], cw_sentences[index][1], cw_flat_conversations[index][-3]))
        else:
            #compliance_list.append(grade_creative_writing_task_adherence(cw_sentences[index][0], cw_sentences[index][1], cw_flat_conversations[index][-1]))
            if (bool(re.search(cw_sentences[index][0] + '\n', cw_flat_conversations_no_metadata[index][-1])) or bool(re.search(cw_sentences[index][0] + ' \n', cw_flat_conversations_no_metadata[index][-1]))) and bool(re.search(cw_sentences[index][1] + '$', cw_flat_conversations_no_metadata[index][-1])):
                compliance_list.append(1)
            else:
                compliance_list.append(0)
        
print(compliance_list[0])


1


In [8]:
# Delete pre-existing file
import os
if os.path.exists('Creative Writing/Non-Compliant Conversations.txt'):
    os.remove('Creative Writing/Non-Compliant Conversations.txt')

# Output all non-compliant conversations to a text file
for index in range(len(cw_flat_conversations_no_metadata)):
    if compliance_list[index] == 0:
        with open('Creative Writing/Non-Compliant Conversations.txt', 'a') as file:
            if 'self_refine' in cw_model_task_methods[index] or 'tree_of_thought' in cw_model_task_methods[index]:
                # Also condition on having cw_sentences[index][0] in the third to last message and cw_sentences[index][1] in the third to last message
                if cw_sentences[index][0] in cw_flat_conversations_no_metadata[index][-3] and cw_sentences[index][1] in cw_flat_conversations_no_metadata[index][-3]:
                    file.write('Conversation ' + str(cw_conversation_numbers[index]) + '\n')
                    file.write(cw_model_task_methods[index] + '\n')
                    file.write(cw_sentences[index][0] + '\n')
                    file.write(cw_sentences[index][1] + '\n')
                    file.write(cw_flat_conversations_no_metadata[index][-3] + '\n')
                    file.write('\n')
            else:
                # Also condition on having cw_sentences[index][0] in the last message and cw_sentences[index][1] in the last message
                if cw_sentences[index][0] in cw_flat_conversations_no_metadata[index][-1] and cw_sentences[index][1] in cw_flat_conversations_no_metadata[index][-1]:
                    file.write('Conversation ' + str(cw_conversation_numbers[index]) + '\n')
                    file.write(cw_model_task_methods[index] + '\n')
                    file.write(cw_sentences[index][0] + '\n')
                    file.write(cw_sentences[index][1] + '\n')
                    file.write(cw_flat_conversations_no_metadata[index][-1] + '\n')
                    file.write('\n')


In [9]:
# Output cw_model_task_methods, cw_conversation_numbers, compliance_list to Excel
df = pd.DataFrame({'model_task_method': cw_model_task_methods, 'conversation_number': cw_conversation_numbers, 'compliance': compliance_list})
df.to_excel('Creative_Writing_Compliance.xlsx', index=False)


In [10]:
# Aggregate compliance by model_task_method - find means
df.drop(columns = ['conversation_number']).groupby('model_task_method').mean().sort_values(by='compliance', ascending=False)


,compliance
model_task_method,
gpt4_cw_manual_few_shot_responses,0.63
gpt4_cw_zero_shot_cot_responses,0.57
gpt4_cw_ape_zero_shot_cot_responses,0.56
gpt4_cw_direct_prompting_responses,0.56
gpt4_cw_least_to_most_responses,0.52
gpt4_cw_manual_cot_responses,0.51
td3_cw_direct_prompting_responses,0.50
gpt4_cw_self_refine_responses,0.48
td3_cw_ape_zero_shot_cot_responses,0.48


In [11]:
# Overall mean
df['compliance'].mean()


0.4275

In [12]:
# Check for missing values of compliance
df['compliance'].isnull().sum()


0